In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml
import html5lib
from unidecode import unidecode

## Fetching armor sets from godofwar.fandom.com

In [2]:
def getHtmlText(url): 
    response = requests.get(url) 
    return response.text

In [3]:
# Create soup
url = 'https://godofwar.fandom.com/wiki/Armor_Sets_(Ragnar%C3%B6k)'
doc = getHtmlText(url)
soup = BeautifulSoup(doc, 'html.parser')

# Create list of sets names with corresponding urls to wiki
armorSetsLinks = []
div = soup.find(class_='mw-parser-output')
uls = div.find_all('ul')[3:11]

for i in uls:
    lis = i.find_all('li')
    for j in lis:
        setName = j.a['title']
        link = 'https://godofwar.fandom.com' + j.a['href']
        armorSetsLinks.append((setName, link))

In [4]:
armorSetsLinks

[('Survival Set', 'https://godofwar.fandom.com/wiki/Survival_Set'),
 ('Fortified Husk Set', 'https://godofwar.fandom.com/wiki/Fortified_Husk_Set'),
 ("Vidar's Set of Might",
  'https://godofwar.fandom.com/wiki/Vidar%27s_Set_of_Might'),
 ('Risen Snow Set', 'https://godofwar.fandom.com/wiki/Risen_Snow_Set'),
 ('Darkdale Set', 'https://godofwar.fandom.com/wiki/Darkdale_Set'),
 ("Niðavellir's Finest Set",
  'https://godofwar.fandom.com/wiki/Ni%C3%B0avellir%27s_Finest_Set'),
 ('Enlightenment Set', 'https://godofwar.fandom.com/wiki/Enlightenment_Set'),
 ('Raven Tears Set', 'https://godofwar.fandom.com/wiki/Raven_Tears_Set'),
 ('Radiance Set', 'https://godofwar.fandom.com/wiki/Radiance_Set'),
 ('Spiritual Set', 'https://godofwar.fandom.com/wiki/Spiritual_Set'),
 ("Sól's Set of Courage",
  'https://godofwar.fandom.com/wiki/S%C3%B3l%27s_Set_of_Courage'),
 ("Lúnda's Lost Set",
  'https://godofwar.fandom.com/wiki/L%C3%BAnda%27s_Lost_Set'),
 ('Guiding Light Set', 'https://godofwar.fandom.com/wiki/

In [5]:
# Convert item level
def getLevel(col):
    list = col.split(' ')
    if '•' in list[-1]:
        return list[1] + '.' + str(list[-1].count('•'))
    else: return list[1]

In [6]:
# Get item properties from dataframe, return a part of final dataframe
def getItemFeatures(table, df_main):
    table = table.copy()
    title = table.columns[0][0]
    itemType = ''

    if ': ' in title:
        split = title.split(': ')
        title = split[1]

        if split[0].lower() == 'chest': 
            itemType = 'Chest Armor'
        elif split[0].lower() == 'wrist': 
            itemType = 'Wrist Armor'
        elif split[0].lower() == 'waist': 
            itemType = 'Waist Armor'

    itemName = title.replace(' +', '')

    table.columns = table.columns.droplevel()
    table.rename(columns={'Stats': 'Lvl', 'Level': 'Lvl'}, inplace=True)
    table.rename(columns={'Acquisition': 'Acquisition/Costs'}, inplace=True)

    perk = '-'
    if table.iat[table.shape[0]-1, 0].lower() == 'perk':
        perk = table.at[table.shape[0]-2, 'Acquisition/Costs'] + ': ' + table.at[table.shape[0]-1, 'Acquisition/Costs']
        table = table[:-2]

    correctRows = range(0, table.shape[0], 2)

    for i in correctRows:
        stats = table.at[i+1, 'Lvl'].split(' ')
        for j in range(1, len(stats), 2):
            table.at[i, stats[j]] = stats[j-1]

    table = table.iloc[correctRows]
    
    table[['Item', 'Type']] = [itemName, itemType]
    table['Perk'] = perk
    table['Lvl'] = table['Lvl'].apply(getLevel)

    return table

In [7]:
# Fetch item information tables from corresponding urls
dfArmor = pd.DataFrame()

for i in armorSetsLinks:
    print(i[0])
    tables = pd.read_html(i[1])

    for j in range(2,min(8, len(tables))):
        try:
            newTable = getItemFeatures(tables[j], dfArmor)
        except Exception as error:
            print("An exception occurred:", error)
        else:
            newTable[['Set', 'Wiki Link']] = i[0], i[1]
            i[1]
            dfArmor = pd.concat([dfArmor, newTable], ignore_index=True, axis=0)

Survival Set
Fortified Husk Set
Vidar's Set of Might
Risen Snow Set
Darkdale Set
Niðavellir's Finest Set
Enlightenment Set
Raven Tears Set
Radiance Set
Spiritual Set
Sól's Set of Courage
Lúnda's Lost Set
Guiding Light Set
Giptumaðr's Set
Máni's Set of Wisdom
Berserker Set
Steinbjörn Set
Fallen Stars Set
Dragon Scaled Set
Undying Pyres Set
Hunter's Set
Fate Breaker Set
Surtr's Scorched Set
Black Bear Set
Spartan Set
Spartan Set (Valhalla)
Armor of Ares (Ragnarök)
Armor of Zeus (Ragnarök)


In [8]:
# Add the missing NG+ Spartan set as its wiki page varies from other sets
spartanSet = pd.DataFrame({'Item': ['Sorrow of Sparta', 'Scars of Sparta', 'Woe of Sparta'], 
                           'Set': ['Spartan Set'] * 3,
                           'Type': ['Chest Armor', 'Wrist Armor', 'Waist Armor'],
                           'Lvl': ['1'] * 3,
                           'Acquisition/Costs': ['Costs 25,000 Hacksilver', 'Costs 16,500 Hacksilver', 'Costs 16,500 Hacksilver'],
                           'Perk': ['-'] * 3,
                           'Wiki Link': ['https://godofwar.fandom.com/wiki/Spartan_Set'] * 3})
dfArmor = pd.concat([dfArmor, spartanSet], ignore_index=True, axis=0)

In [9]:
# Format the dataframe with all armor parts
dfArmor = dfArmor[['Item', 'Set', 'Type', 'Lvl','STR', 'DEF', 'RNC', 'VIT', 'CLD', 'LCK', 'Acquisition/Costs', 'Wiki Link', 'Perk']]
dfArmor = dfArmor.astype({'STR': 'float', 'DEF': 'float', 'RNC': 'float', 'VIT': 'float', 'CLD': 'float', 'LCK': 'float'})
dfArmor.dtypes

Item                  object
Set                   object
Type                  object
Lvl                   object
STR                  float64
DEF                  float64
RNC                  float64
VIT                  float64
CLD                  float64
LCK                  float64
Acquisition/Costs     object
Wiki Link             object
Perk                  object
dtype: object

In [10]:
# Dataframe with armor parts
dfArmor

,Item,Set,Type,Lvl,STR,DEF,RNC,VIT,CLD,LCK,Acquisition/Costs,Wiki Link,Perk
0,Shoulder Guard of Survival,Survival Set,Chest Armor,1,NaN,2.0,NaN,NaN,NaN,NaN,Default armor,https://godofwar.fandom.com/wiki/Survival_Set,-
1,Shoulder Guard of Survival,Survival Set,Chest Armor,2,12.0,12.0,NaN,NaN,NaN,NaN,"Upgrade: - 1,000 Hacksilver - 5 Forged Iron",https://godofwar.fandom.com/wiki/Survival_Set,-
2,Shoulder Guard of Survival,Survival Set,Chest Armor,3,20.0,20.0,NaN,NaN,NaN,NaN,"Upgrade: - 2,000 Hacksilver - 10 Forged Iron",https://godofwar.fandom.com/wiki/Survival_Set,-
3,Shoulder Guard of Survival,Survival Set,Chest Armor,4,28.0,28.0,NaN,NaN,NaN,NaN,"Upgrade: - 6,000 Hacksilver - 4 Dwarven Steel",https://godofwar.fandom.com/wiki/Survival_Set,-
4,Shoulder Guard of Survival,Survival Set,Chest Armor,5,36.0,36.0,NaN,NaN,NaN,NaN,"Upgrade: - 10,000 Hacksilver - 8 Dwarven Steel",https://godofwar.fandom.com/wiki/Survival_Set,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,Gauntlets of Zeus+,Armor of Zeus (Ragnarök),,10,125.0,NaN,50.0,NaN,NaN,NaN,"Favor: Remnants of Asgard - The Barrens, Alfhe...",https://godofwar.fandom.com/wiki/Armor_of_Zeus...,Divine Edge: Increases Kratos' Melee and Runic...
751,War Belt of Zeus+,Armor of Zeus (Ragnarök),,10,NaN,125.0,50.0,NaN,NaN,NaN,Favor: Remnants of Asgard - Eastern Barri Wood...,https://godofwar.fandom.com/wiki/Armor_of_Zeus...,Divine Edge: Increases Kratos' Melee and Runic...
752,Sorrow of Sparta,Spartan Set,Chest Armor,1,NaN,NaN,NaN,NaN,NaN,NaN,"Costs 25,000 Hacksilver",https://godofwar.fandom.com/wiki/Spartan_Set,-
753,Scars of Sparta,Spartan Set,Wrist Armor,1,NaN,NaN,NaN,NaN,NaN,NaN,"Costs 16,500 Hacksilver",https://godofwar.fandom.com/wiki/Spartan_Set,-


## Fetching weapon attachments from godofwar.fandom.com and game8.co

In [11]:
# Fetch attachments links from wiki
url = 'https://godofwar.fandom.com/wiki/Attachments_(Ragnar%C3%B6k)'
doc = getHtmlText(url)
soup = BeautifulSoup(doc, 'html.parser')

# Create a dictionary with names of attachments with corresponding urls
weapons = ['Axe', 'Blades', 'Spear', 'Shield']
div = soup.find(class_='mw-parser-output')
uls = div.findAll('ul')[2:6]

linkDict = {'Axe': pd.Series(), 
                'Blades': pd.Series(), 
                'Spear': pd.Series(), 
                'Shield': pd.Series()}

for i in zip(weapons, uls):
    lis = i[1].find_all('li')
    for j in lis:
        link = 'https://godofwar.fandom.com' + j.a['href']
        #attachmentsLinks.append((i[0], j.a.text, link))
        linkDict[i[0]].at[j.a.text] = link

linkDict

{'Axe': Wooden Knob                          https://godofwar.fandom.com/wiki/Wooden_Knob
 The Furious Maul                https://godofwar.fandom.com/wiki/The_Furious_Maul
 Grip of Darkdale                https://godofwar.fandom.com/wiki/Grip_of_Darkdale
 Fortified Frost Knob            https://godofwar.fandom.com/wiki/Fortified_Fro...
 Haur's Lucky Knob               https://godofwar.fandom.com/wiki/Haur%27s_Luck...
 Grip of Weighted Recovery       https://godofwar.fandom.com/wiki/Grip_of_Weigh...
 Stonecutter's Knob              https://godofwar.fandom.com/wiki/Stonecutter%2...
 Grip of Healing Harmony         https://godofwar.fandom.com/wiki/Grip_of_Heali...
 Grip of the Fallen Alchemist    https://godofwar.fandom.com/wiki/Grip_of_the_F...
 Grip of Radiant Reflection      https://godofwar.fandom.com/wiki/Grip_of_Radia...
 Banahogg Knob                      https://godofwar.fandom.com/wiki/Banahogg_Knob
 Runic Hailstone Knob            https://godofwar.fandom.com/wiki/Runic_Hailsto.

In [12]:
# For every attachment get a url to ralated game8.co page
link = 'https://game8.co/games/God-of-War-Ragnarok/archives/395838'
game8List = pd.read_html(link, extract_links='all')

# Create dis=ctionary with all attachments
perkLinkDict = {'Axe': pd.Series(dict(game8List[1]['Attachment', None].tolist())), 
                'Blades': pd.Series(dict(game8List[2]['Attachment', None].tolist())), 
                'Spear': pd.Series(dict(game8List[4]['Attachment', None].tolist())), 
                'Shield': pd.Series(dict(game8List[3]['Attachment', None].tolist()))}

perkLinkDict

{'Axe': Banahogg Knob                   https://game8.co/games/God-of-War-Ragnarok/arc...
 Fortified Frost Knob            https://game8.co/games/God-of-War-Ragnarok/arc...
 Grip of Healing Harmony         https://game8.co/games/God-of-War-Ragnarok/arc...
 Grip of Radiant Reflection      https://game8.co/games/God-of-War-Ragnarok/arc...
 Grip of the Fallen Alchemist    https://game8.co/games/God-of-War-Ragnarok/arc...
 Grip of the Nine Realms         https://game8.co/games/God-of-War-Ragnarok/arc...
 Grip of Weighted Recovery       https://game8.co/games/God-of-War-Ragnarok/arc...
 Haur's Lucky Knob               https://game8.co/games/God-of-War-Ragnarok/arc...
 Runic Hailstone Knob            https://game8.co/games/God-of-War-Ragnarok/arc...
 Stonecutter's Knob              https://game8.co/games/God-of-War-Ragnarok/arc...
 The Furious Maul                https://game8.co/games/God-of-War-Ragnarok/arc...
 Wooden Knob                     https://game8.co/games/God-of-War-Ragnarok/arc.

In [13]:
# Fetch attachments from wiki and perks from game8.co
dfAttach = pd.DataFrame()

for weapon in weapons:
    series = linkDict[weapon]

    for itemName, url in series.items():
        tables = pd.read_html(url)[1:3]
        print(itemName)
    
        newTable = pd.DataFrame()
    
        for j in tables:
            fetchedTable = getItemFeatures(j, dfAttach)
            newTable = pd.concat([newTable, fetchedTable], ignore_index=True, axis=0)
    
        try:
            perkLink = perkLinkDict[weapon][unidecode(itemName)]
        except Exception as error:
            print("An exception occurred:", error)
            perk = 'Unable to fetch'
        else:
            perk = pd.read_html(perkLink)[0].iat[-1, 0]
    
        newTable[['Type', 'Item', 'Perk', 'Wiki Link']] = weapon, itemName, perk, url
        dfAttach = pd.concat([dfAttach, newTable], ignore_index=True, axis=0)

Wooden Knob
The Furious Maul
Grip of Darkdale
An exception occurred: 'Grip of Darkdale'
Fortified Frost Knob
Haur's Lucky Knob
Grip of Weighted Recovery
Stonecutter's Knob
Grip of Healing Harmony
Grip of the Fallen Alchemist
Grip of Radiant Reflection
Banahogg Knob
Runic Hailstone Knob
Grip of the Nine Realms
Steel Handles
Deadly Obsidian Handles
Darkdale Handles
An exception occurred: 'Darkdale Handles'
Cursed Empress Handles
Pommels of the Undying Spark
Radiant Warden Handles
Pommels of Agile Deceit
Pommels of Brutal Might
Hardened War Handles
Pommels of the True Flame
Pommels of the Nine Realms
Pommels of Burning Unity
Luminous Recovery Handles
Soldier's Sauroter
Lion's Roar Sauroter
Charging Assault Sauroter
Mighty Olympic Sauroter
Hind of Attuned Elements
Battle-Scarred Sauroter
Hind of Weightless Recovery
Hind of Volatile Might
Hind of the Nine Realms
Hind of the Four Winds
Warrior's Echo Sauroter
Hind of Deadly Vitality
Rönd of Volition
Rönd of Aggravation
Rönd of Expedition
Rön

In [14]:
# Adding the missing "Darkdale" attachments 
darkdaleGripPerk = 'DEADLY AWAKENING Killing enemies with a Frost Awaken (Hold Triangle) attack causes a Frost shockwave and re-Frost Awakens the Axe.'
darkdaleHandlesPerk = 'DEADLY FLAMES Killing enemies with a Flame Whiplash (Mash Triangle) attack causes a Burn shockwave and increases Burn status inflicted for a duration.'
dfAttach.loc[dfAttach['Item'] == 'Darkdale Handles', 'Perk'] = darkdaleHandlesPerk
dfAttach.loc[dfAttach['Item'] == 'Grip of Darkdale', 'Perk'] = darkdaleGripPerk

In [15]:
# Rearrange the dataframe with attachments
dfAttach = dfAttach[['Item', 'Type', 'Lvl','STR', 'DEF', 'RNC', 'VIT', 'CLD', 'LCK', 'Acquisition/Costs', 'Wiki Link', 'Perk']]
dfAttach = dfAttach.astype({'STR': 'float', 'DEF': 'float', 'RNC': 'float', 'VIT': 'float', 'CLD': 'float', 'LCK': 'float'})
dfAttach.dtypes

Item                  object
Type                  object
Lvl                   object
STR                  float64
DEF                  float64
RNC                  float64
VIT                  float64
CLD                  float64
LCK                  float64
Acquisition/Costs     object
Wiki Link             object
Perk                  object
dtype: object

In [16]:
dfAttach

,Item,Type,Lvl,STR,DEF,RNC,VIT,CLD,LCK,Acquisition/Costs,Wiki Link,Perk
0,Wooden Knob,Axe,1,NaN,NaN,NaN,NaN,NaN,NaN,Start the game.,https://godofwar.fandom.com/wiki/Wooden_Knob,This attachment does not grant any perk.
1,Wooden Knob,Axe,2,4.0,NaN,NaN,NaN,NaN,NaN,Upgrade: - 400 Hacksilver - 3 Rawhide,https://godofwar.fandom.com/wiki/Wooden_Knob,This attachment does not grant any perk.
2,Wooden Knob,Axe,3,8.0,NaN,NaN,NaN,NaN,NaN,Upgrade: - 800 Hacksilver - 6 Rawhide,https://godofwar.fandom.com/wiki/Wooden_Knob,This attachment does not grant any perk.
3,Wooden Knob,Axe,4,11.0,NaN,NaN,NaN,NaN,NaN,Upgrade: - 2400 Hacksilver - 2 Stonewood,https://godofwar.fandom.com/wiki/Wooden_Knob,This attachment does not grant any perk.
4,Wooden Knob,Axe,5,15.0,NaN,NaN,NaN,NaN,NaN,Upgrade: - 4000 Hacksilver - 5 Stonewood,https://godofwar.fandom.com/wiki/Wooden_Knob,This attachment does not grant any perk.
...,...,...,...,...,...,...,...,...,...,...,...,...
476,Rönd of Obliteration,Shield,9,11.0,11.0,14.0,14.0,14.0,14.0,"Upgrade: 14,000 Hacksilver - 30 Dust of Realms...",https://godofwar.fandom.com/wiki/R%C3%B6nd_of_...,SHARDS OF THE VALKYRIE Hold Block for several ...
477,Rönd of Obliteration,Shield,9.1,13.0,13.0,16.0,16.0,16.0,16.0,"Craft: - 15,000 Hacksilver - 10 Skap Slag",https://godofwar.fandom.com/wiki/R%C3%B6nd_of_...,SHARDS OF THE VALKYRIE Hold Block for several ...
478,Rönd of Obliteration,Shield,9.2,14.0,14.0,18.0,18.0,18.0,18.0,"Upgrade: - 16,000 Hacksilver - 20 Skap Slag - ...",https://godofwar.fandom.com/wiki/R%C3%B6nd_of_...,SHARDS OF THE VALKYRIE Hold Block for several ...
479,Rönd of Obliteration,Shield,9.3,16.0,16.0,20.0,20.0,20.0,20.0,"Upgrade: - 18,000 Hacksilver - 30 Skap Slag - ...",https://godofwar.fandom.com/wiki/R%C3%B6nd_of_...,SHARDS OF THE VALKYRIE Hold Block for several ...


## Fetching enchantments from godofwar.fandom.com

In [17]:
# Fetch enthantments table from wiki
url = 'https://godofwar.fandom.com/wiki/Enchantments_(Ragnar%C3%B6k)?so=search'
tables = pd.read_html(url)
# Create dataframe with enchantments
dfEnch = pd.DataFrame()

In [18]:
# Add the "realm" class enchantments to dataframe
for i in tables[0:9]:
    table = i.copy()
    enchSet = table.columns[0][0].split(' ')[0]
    table.columns = table.columns.droplevel()
    setPerk = 'When wearing 3: ' + table.iloc[-2, 1]

    correctRows = range(0, table.shape[0]-2, 2)
    
    for i in correctRows:
        stats = table.at[i+1, 'Enchantment Name'].split(' ')
        for j in range(1, len(stats), 2):
            table.at[i, stats[j]] = stats[j-1]
    
    table = table.iloc[correctRows]
    table[['Set', 'Perk', 'Stat Requirement']] = enchSet, setPerk, '-'
    table.rename(columns={'Enchantment Name': 'Item'}, inplace=True)

    dfEnch = pd.concat([dfEnch, table], ignore_index=True, axis=0)

dfEnch

,Item,Acquisition,CLD,LCK,VIT,RNC,DEF,Set,Perk,Stat Requirement,STR
0,Midgard's Endurance,Defeat Níðhögg,10,NaN,NaN,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
1,Midgard's Honour,Craft: - 1000 Hacksilver - Commence your searc...,5,8,NaN,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
2,Midgard's Virtue,Craft: - 1500 Hacksilver - Return to Sindri's ...,4,NaN,9,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
3,Midgard's Justice,Treasure Hunt: Viking's Gift,3,NaN,NaN,12,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
4,Midgard's Bravery,Nornir Chest: 31st Nornir Chest opened,2,NaN,NaN,NaN,20,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
5,Asgard's Fortitude,Berserker: Harðrefill the Callous,3,NaN,NaN,NaN,NaN,Asgard,When wearing 3: Increases the damage of all of...,-,12
6,Asgard's Might,Berserker: Beigaðr the Feared,3,NaN,NaN,NaN,12,Asgard,When wearing 3: Increases the damage of all of...,-,NaN
7,Asgard's Security,Berserkers: Svipdagr the Cold & The Sisters of...,3,NaN,NaN,12,NaN,Asgard,When wearing 3: Increases the damage of all of...,-,NaN
8,Asgard's Justice,Treasure Hunt: A Scar is Born,3,12,NaN,NaN,NaN,Asgard,When wearing 3: Increases the damage of all of...,-,NaN
9,Asgard's Virtue,Shop Coffin: Return the moon to the sky,3,NaN,3,NaN,7,Asgard,When wearing 3: Increases the damage of all of...,-,7


In [19]:
# Separate the "burden" class enchantments, as their table format differs from the rest
burdensList = tables.pop(52)
burdensList

Burdens                                                   
         Enchantment Name                                               Perk
0          Burden of Stun                                     Disrupted Stun
1          Burden of Stun                               Reduces Stun damage.
2         Burden of Runic                                    Disrupted Runic
3         Burden of Runic                       Reduces Runic Attack damage.
4      Burden of Momentum                                 Disrupted Momentum
5      Burden of Momentum  Weapon damage is reduced unless the Permafrost...
6      Burden of Elements                                 Disrupted Elements
7      Burden of Elements  Decreases Resistance to all Status Effects. De...
8   Burden of Restoration                              Disrupted Restoration
9   Burden of Restoration  Decreases the healing from Healthstones, Rage,...
10     Burden of Weaponry                                 Disrupted Weaponry
11     Burden of Weaponry  Reduces the damage and stagger potential of al...
12         Burden of Rage                                     Disrupted Rage
13         Burden of Rage        Decreases the rate at which Rage is gained.
14        Burden of Guard                                    Disrupted Guard
15        Burden of Guard  Kratos now takes damage and loses Rage when Bl...
16      Burden of Evasion                                  Disrupted Evasion
17      Burden of Evasion  Dodge Rolling (Double Tap X) inflicts the Fros...
18            Acquisition  Craft: - 10,000 Hacksilver each - Visit Sindri...
19            Description  "Enchantments for the Spartan wishing to chall...

In [20]:
# Complement dataframe with the rest of enchantments
for i in tables[9:61]:
    table = i.copy()
    item = table.columns[0]
    table.set_index(item, inplace=True)
    acquisition = table.loc['Acquisition'].iat[0]
    
    statsDict = {}
    if 'Stats' in table.index:
        stats = table.loc['Stats'].iat[0].split(' ')
        for j in range(1, len(stats), 2):
            statsDict[stats[j]] = stats[j-1]

    if 'Perk' in table.index:
        perk = table.loc['Perk'].iat[0,0] + ': ' + table.loc['Perk'].iat[1,0]
    else:
        perk = '-'

    if 'Stat Requirement' in table.index:
        requirements = table.loc['Stat Requirement'].iat[0]
    else:
        requirements = '-'

    dfDict = {'Item': item, 'Perk': perk, 'Acquisition': acquisition, 'Stat Requirement': requirements, 'Set': '-'}
    dfDict.update(statsDict)

    dfEnch = pd.concat([dfEnch, pd.DataFrame([dfDict])], ignore_index=True, axis=0)

dfEnch

,Item,Acquisition,CLD,LCK,VIT,RNC,DEF,Set,Perk,Stat Requirement,STR
0,Midgard's Endurance,Defeat Níðhögg,10,NaN,NaN,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
1,Midgard's Honour,Craft: - 1000 Hacksilver - Commence your searc...,5,8,NaN,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
2,Midgard's Virtue,Craft: - 1500 Hacksilver - Return to Sindri's ...,4,NaN,9,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
3,Midgard's Justice,Treasure Hunt: Viking's Gift,3,NaN,NaN,12,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
4,Midgard's Bravery,Nornir Chest: 31st Nornir Chest opened,2,NaN,NaN,NaN,20,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...
92,Lífdagar,Berserker: Hvítserkr the Bold,NaN,NaN,30,NaN,5,-,-,-,NaN
93,Garpr,Berserker: Haklangr the Bearded,NaN,20,NaN,NaN,NaN,-,-,-,20
94,Mannvit,Berserker: Svipdagr the Cold and the Sisters o...,20,NaN,NaN,20,NaN,-,-,-,NaN
95,Vald,Berserker: Hjalti the Stolid,30,NaN,NaN,NaN,NaN,-,-,-,5


In [21]:
# Complement dataframe with the "burden" class enchantments
burdens = burdensList.copy()
burdens.columns = burdens.columns.droplevel()
burdens.rename(columns={'Enchantment Name': 'Item'}, inplace=True)

acquisition = "Craft: - 10,000 Hacksilver - Visit Sindri's House for the first time."
correctRows = range(0, burdens.shape[0]-2, 2)

for i in correctRows:
    burdens.at[i, 'Perk'] = burdens.at[i, 'Perk'] + ': ' + burdens.at[i+1, 'Perk']

burdens = burdens.iloc[correctRows]
burdens[['Acquisition', 'Set', 'Stat Requirement']] = acquisition, '-', '-'

dfEnch = pd.concat([dfEnch, burdens], ignore_index=True, axis=0)

dfEnch

,Item,Acquisition,CLD,LCK,VIT,RNC,DEF,Set,Perk,Stat Requirement,STR
0,Midgard's Endurance,Defeat Níðhögg,10,NaN,NaN,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
1,Midgard's Honour,Craft: - 1000 Hacksilver - Commence your searc...,5,8,NaN,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
2,Midgard's Virtue,Craft: - 1500 Hacksilver - Return to Sindri's ...,4,NaN,9,NaN,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
3,Midgard's Justice,Treasure Hunt: Viking's Gift,3,NaN,NaN,12,NaN,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
4,Midgard's Bravery,Nornir Chest: 31st Nornir Chest opened,2,NaN,NaN,NaN,20,Midgard,When wearing 3: Increases the damage of all of...,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...
101,Burden of Restoration,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",NaN,NaN,NaN,NaN,NaN,-,Disrupted Restoration: Decreases the healing f...,-,NaN
102,Burden of Weaponry,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",NaN,NaN,NaN,NaN,NaN,-,Disrupted Weaponry: Reduces the damage and sta...,-,NaN
103,Burden of Rage,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",NaN,NaN,NaN,NaN,NaN,-,Disrupted Rage: Decreases the rate at which Ra...,-,NaN
104,Burden of Guard,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",NaN,NaN,NaN,NaN,NaN,-,Disrupted Guard: Kratos now takes damage and l...,-,NaN


In [22]:
dfEnch = dfEnch[['Item', 'Set', 'STR', 'DEF', 'RNC', 'VIT', 'CLD', 'LCK', 'Acquisition', 'Stat Requirement', 'Perk']]
dfEnch = dfEnch.astype({'STR': 'float', 'DEF': 'float', 'RNC': 'float', 'VIT': 'float', 'CLD': 'float', 'LCK': 'float'})
dfEnch.dtypes
dfEnch

,Item,Set,STR,DEF,RNC,VIT,CLD,LCK,Acquisition,Stat Requirement,Perk
0,Midgard's Endurance,Midgard,NaN,NaN,NaN,NaN,10.0,NaN,Defeat Níðhögg,-,When wearing 3: Increases the damage of all of...
1,Midgard's Honour,Midgard,NaN,NaN,NaN,NaN,5.0,8.0,Craft: - 1000 Hacksilver - Commence your searc...,-,When wearing 3: Increases the damage of all of...
2,Midgard's Virtue,Midgard,NaN,NaN,NaN,9.0,4.0,NaN,Craft: - 1500 Hacksilver - Return to Sindri's ...,-,When wearing 3: Increases the damage of all of...
3,Midgard's Justice,Midgard,NaN,NaN,12.0,NaN,3.0,NaN,Treasure Hunt: Viking's Gift,-,When wearing 3: Increases the damage of all of...
4,Midgard's Bravery,Midgard,NaN,20.0,NaN,NaN,2.0,NaN,Nornir Chest: 31st Nornir Chest opened,-,When wearing 3: Increases the damage of all of...
...,...,...,...,...,...,...,...,...,...,...,...
101,Burden of Restoration,-,NaN,NaN,NaN,NaN,NaN,NaN,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",-,Disrupted Restoration: Decreases the healing f...
102,Burden of Weaponry,-,NaN,NaN,NaN,NaN,NaN,NaN,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",-,Disrupted Weaponry: Reduces the damage and sta...
103,Burden of Rage,-,NaN,NaN,NaN,NaN,NaN,NaN,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",-,Disrupted Rage: Decreases the rate at which Ra...
104,Burden of Guard,-,NaN,NaN,NaN,NaN,NaN,NaN,"Craft: - 10,000 Hacksilver - Visit Sindri's Ho...",-,Disrupted Guard: Kratos now takes damage and l...


## Fetching shields from godofwar.fandom.com

In [23]:
# Shield links
shieldsUrls = ['https://godofwar.fandom.com/wiki/Guardian_Shield',
               'https://godofwar.fandom.com/wiki/Dauntless_Shield',
               'https://godofwar.fandom.com/wiki/Stone_Wall_Shield',
               'https://godofwar.fandom.com/wiki/Shatter_Star_Shield',
               'https://godofwar.fandom.com/wiki/Onslaught_Shield',
               'https://godofwar.fandom.com/wiki/Spartan_Aspis']

In [24]:
# Fetch shields properties from wiki
def getShieldLevel(col):
    list = col.replace(':', '').split(' ')
    if '•' in list[-1]:
        return list[1] + '.' + str(list[-1].count('•'))
    else: return list[1]

def fetchShield(table, url, df):
    table = table.copy()
    title = table.columns[0][0]
    itemName = title.replace(' +', '')
    table.columns = table.columns.droplevel()
    table.rename(columns={'Stats': 'Lvl'}, inplace=True)

    if 'Acquisition' in table.columns:
        table.rename(columns={'Acquisition': 'Acquisition/Costs'}, inplace=True)

    lastRow = table.loc[table['Lvl']=='Shield Strike'].index[0]
    correctRows = range(0, lastRow, 2)

    perk = table.loc[table['Lvl']=='Shield Strike'].iat[0,1] + ': ' + table.loc[table['Lvl']=='Shield Strike'].iat[1,1]
    table = table[:-2]
    
    for i in correctRows:
        stats = table.at[i+1, 'Lvl'].split(' ')
        for j in range(1, len(stats), 2):
            table.at[i, stats[j]] = stats[j-1]
    
    table = table.iloc[correctRows]
        
    table['Lvl'] = table['Lvl'].apply(getLevel)
    table[['Perk', 'Item', 'Wiki Link']] = perk, itemName, url

    df = pd.concat([df, table], ignore_index=True, axis=0)
    
    return df

In [25]:
# Create dataframe with shields
dfShields = pd.DataFrame()

for url in shieldsUrls:
    itemName = url.split('/')[-1].replace('_', ' ')
    tables = pd.read_html(url)

    print(itemName)

    for i in tables:
        if type(i.columns[0])==tuple and len(i.columns[0])==2:
            if itemName in i.columns[0][0]:
                dfShields = fetchShield(i, url, dfShields)

Guardian Shield
Dauntless Shield
Stone Wall Shield
Shatter Star Shield
Onslaught Shield
Spartan Aspis


In [26]:
dfShields['STR'] = np.nan
dfShields = dfShields[['Item', 'Lvl', 'STR', 'DEF', 'RNC', 'VIT', 'CLD', 'LCK', 'Acquisition/Costs', 'Wiki Link', 'Perk']]
dfShields.astype({'STR': 'float', 'DEF': 'float', 'RNC': 'float', 'VIT': 'float', 'CLD': 'float', 'LCK': 'float'})

dfShields

,Item,Lvl,STR,DEF,RNC,VIT,CLD,LCK,Acquisition/Costs,Wiki Link,Perk
0,Guardian Shield,4,NaN,55,NaN,NaN,NaN,NaN,"Acquire from Sindri, before commencing your se...",https://godofwar.fandom.com/wiki/Guardian_Shield,Shield Check: Double Tap L1 to quickly Shield ...
1,Guardian Shield,5,NaN,70,NaN,NaN,NaN,NaN,Upgrade: - 5000 Hacksilver - 6 Honed Metal,https://godofwar.fandom.com/wiki/Guardian_Shield,Shield Check: Double Tap L1 to quickly Shield ...
2,Guardian Shield,6,NaN,85,NaN,NaN,NaN,NaN,Upgrade: - 7500 Hacksilver - 2 Luminous Alloy,https://godofwar.fandom.com/wiki/Guardian_Shield,Shield Check: Double Tap L1 to quickly Shield ...
3,Guardian Shield,7,NaN,100,NaN,NaN,NaN,NaN,Upgrade: - 11000 Hacksilver - 4 Luminous Alloy,https://godofwar.fandom.com/wiki/Guardian_Shield,Shield Check: Double Tap L1 to quickly Shield ...
4,Guardian Shield,8,NaN,115,NaN,NaN,NaN,NaN,Upgrade: - 15000 Hacksilver - 6 Luminous Alloy...,https://godofwar.fandom.com/wiki/Guardian_Shield,Shield Check: Double Tap L1 to quickly Shield ...
...,...,...,...,...,...,...,...,...,...,...,...
58,Spartan Aspis,9,NaN,130,NaN,NaN,NaN,NaN,Upgrade: - 20000 Hacksilver - 10 Luminous Alloy,https://godofwar.fandom.com/wiki/Spartan_Aspis,Shield Check: Double Tap L1 to quickly Shield ...
59,Spartan Aspis,9.1,NaN,145,NaN,NaN,NaN,NaN,"Craft: - 21,000 Hacksilver - 10 Skap Slag",https://godofwar.fandom.com/wiki/Spartan_Aspis,Shield Check: Double Tap L1 to quickly Shield ...
60,Spartan Aspis,9.2,NaN,160,NaN,NaN,NaN,NaN,"Upgrade: - 22,000 Hacksilver - 20 Skap Slag - ...",https://godofwar.fandom.com/wiki/Spartan_Aspis,Shield Check: Double Tap L1 to quickly Shield ...
61,Spartan Aspis,9.3,NaN,175,NaN,NaN,NaN,NaN,"Upgrade: - 23,000 Hacksilver - 30 Skap Slag - ...",https://godofwar.fandom.com/wiki/Spartan_Aspis,Shield Check: Double Tap L1 to quickly Shield ...


## Export dataframes to a .xlsx file

In [27]:
# Export dataframes to a .xls file
path = 'GoW_mastersheet.xlsx'

with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
    dfArmor.to_excel(writer, sheet_name='Armor', index=False, freeze_panes=(1,1))
    dfAttach.to_excel(writer, sheet_name='Attachments', index=False, freeze_panes=(1,1))
    dfEnch.to_excel(writer, sheet_name='Enchantments', index=False, freeze_panes=(1,1))
    dfShields.to_excel(writer, sheet_name='Shields', index=False, freeze_panes=(1,1))